In [6]:
from torch.utils.data import Dataset
import numpy as np
from torch.nn.functional import one_hot
import torch

class M100ImageDataset(Dataset):
    def __init__(self, *, data_path, train=True):
        # Loading data from the 'mnist_compressed.npz' file
        data = np.load(data_path)
        # Reading variables containing the data
        X_test, y_test, X_train, y_train =  data['test_images'], data['test_labels'], data['train_images'], data['train_labels']

        if train:
            self.data = torch.tensor(X_train.astype('float32') / 255.)
            self.labels = one_hot(torch.tensor(y_train), num_classes=100)
        else:
            self.data = torch.tensor(X_test.astype('float32') / 255.)
            self.labels = one_hot(torch.tensor(y_test), num_classes=100)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [7]:
from torch.utils.data import DataLoader

m100_path = '../data/mnist100/mnist_compressed.npz'

training_data = M100ImageDataset(data_path=m100_path, train=True)
test_data = M100ImageDataset(data_path=m100_path, train=False)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)